# Chunk 기반 RAG

청크단위 RAG 베이스라인 만들기

LLM 플로우

- OCR로 text 생성 → 청크단위로 분할 → 하나의 LLM(w RAG)을 이용해서 하나의 답변 생성 → API 변환
내용

- 각 청크별로 RAG 검색하여 (n x 청크 수)개의 검색 자료를 이용해서 하나의 LLM 답변을 생성함 (n: 검색 수(default=1))

https://github.com/boostcampaitech7/level4-recsys-finalproject-hackathon-recsys-04-lv3/issues/42

In [ ]:
import time
from glob import glob

# from app.core.config import settings
from fastapi import HTTPException
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from pinecone import Pinecone, ServerlessSpec
# from utils import logging




# import os
# import requests
from dotenv import load_dotenv

# from langchain_upstage import UpstageDocumentParseLoader
# from langchain_upstage import UpstageEmbeddings
# from langchain_upstage import ChatUpstage

# from pinecone import Pinecone, ServerlessSpec
# from langchain_pinecone import PineconeVectorStore
# # from langchain_community.vectorstores import FAISS
# from langchain_community.document_loaders import PyMuPDFLoader
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# from langchain_teddynote.community.pinecone import create_index

from rag_utils import logging

# API 키 정보 로드
load_dotenv()

In [ ]:
ocr_txt = '''
별의 형성 성운 단계:성운(가스와 먼지 구름)이 온도 하강으로 인해 수축이 시작.중심부가 차가워지며 밀도가 높아지고, 원시별이 형성됨. 
원시별 단계:핵융합 반응이 바로 시작되며 중심부 온도가 약 1천만 K로 상승. 
주계열성 단계 핵융합 시작:중심부 온도가 약 10억 K에 도달하면 수소가 헬륨으로 융합되며 에너지를 방출. 
특징:별은 내부의 중력과 외부로 나가는 방사압이 균형을 이루지 못하며 진동을 반복.질량이 클수록 수명이 길어지고 더 밝으며 온도는 낮아짐. 
거성 또는 초거성 단계 핵융합 종료 후 부풀어오름:중심부의 수소가 모두 소모되면 별 전체가 서서히 수축하며 적색 거성 또는 초거성이 됨. 
핵융합 반복:질량이 큰 별은 헬륨 이후에도 탄소, 산소, 규소 등을 융합하며 금까지 생성. 
별의 최후: 질량에 따른 진화(1) 질량이 작은 별 (태양과 비슷한 별) 행성상 성운:핵융합 종료 후 바깥층이 우주로 방출되지 않고 중심부로 흡수됨. 
백색왜성:중심부가 폭발하여 작은 규모의 초신성이 형성됨.(2) 질량이 큰 별 초신성 폭발:핵융합으로 헬륨까지만 생성된 후 더 이상 에너지를 방출할 수 없어 중력이 붕괴. 
중성자별:초신성 폭발 후 남은 별의 중심부가 수소로 이루어진 별로 변환됨. 
블랙홀:질량이 큰 별은 외곽층이 모두 날아가며, 작은 블랙홀이 형성됨. 
별의 진화 요약 
질량이 작은 별 → 초거성 → 행성상 성운 → 백색왜성 질량이 큰 별 → 적색 거성 → 초신성 → (질량에 따라) 행성상 성운 또는 중성자별
'''


ocr_txt = """형법
범죄 : 국가가 형벌로 처벌할 수 있는 행위
공권력 : 국가나 공공단체가 우월한 의사의 주체로서 국민에 대하여 명령하거나 강제하는 권력
죄형 법정주의
-`법률이 없으면 범죄도 없고 형벌도 없다' 라는 근대 형법의 기본원리
국가가 범죄를 정하고 형벌을 부과하는 효과
1 응보론
2 일반 예방론
3 특수 예방론
형법의 기능
1 법익 보호
2 국민을 범죄로부터 보도
3 국민의 권리 보장 
"""

ocr_txt = """범죄의 성립요건
- 구체적인 행위가 없어도 범죄는 성립된다
- 실수로 법에서 금지하는 결과를 초래했을 때 법에서는 이를 고의 라고 한다
- 범죄자가 의도한 대로 범리가 완전히 성립한 경우는 미수범 이라고 한다
위법성 조각 사유 종류
- 정당 방위
- 긴급 피난
- 자구행위
- 피해자의 승낙
- 정당 행위
"""


## 텍스트를 청크 단위로 분할하기 - 시멘틱 청커(SemanticChunker)

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
# from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings

embeddings = UpstageEmbeddings(model="embedding-passage")

text_splitter = SemanticChunker(
    embeddings=embeddings,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
)

In [ ]:
chunks = text_splitter.split_text(ocr_txt)

print(type(chunks))
print(len(chunks))

In [ ]:
for i, chunk in enumerate(chunks):
    print("="*10)
    print(f"[Chunk {i}]")
    print(chunk)

## 청크별 RAG Retrieval

In [ ]:
LANGSMITH_PROJECT_NAME = "dev-02"
PINECONE_INDEX_NAME = "dev-02"

# LangSmith 시작
# logging.langsmith(settings.LANGSMITH_PROJECT_NAME)
# db_index_name = settings.PINECONE_INDEX_NAME
logging.langsmith(LANGSMITH_PROJECT_NAME)
db_index_name = PINECONE_INDEX_NAME

pc = Pinecone()
index = pc.Index(db_index_name)
embeddings = UpstageEmbeddings(model="embedding-passage")
vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

# 검색기(Retriever) 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# 청크별 검색
retrieved_docs = []    # Document list
retrieved_ids = []    # RAG id list
print("chunks len: ", len(chunks))
for chunk in chunks:
    docs = retriever.invoke(chunk)

    for doc in docs:
        doc_id = doc.id
        if not doc_id in retrieved_ids:
            retrieved_ids.append(doc_id)
            retrieved_docs.append(doc)

print("Retrieved docs len: ", len(retrieved_docs))
print("Retrieved ids len: ", len(retrieved_ids))


In [ ]:
retrieved_ids

In [ ]:
retrieved_docs

## LLM 답변 생성

In [ ]:
# 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """너는 입력을 보고 틀린 부분에 대해서 피드백을 주는 선생님이야
아래 추가 조건과 정보를 참고해서 피드백 해줘
추가 조건
1. 참고한 정보의 이름과 페이지 알려줘
2. 입력에 틀린 부분이 하나도 없을 경우에만 칭찬 한 문장 해줘
3. 다른 미사여구 없이 본론부터 말해줘

#정보:
{context}

#입력:
{question}

#답:"""
)

# 언어모델(LLM) 생성
llm = ChatUpstage(model="solar-pro", temperature=0.2)

chain = prompt | llm | StrOutputParser()

response = chain.invoke({"context": retrieved_docs, "question": ocr_txt})

print(response)

# 함수

In [ ]:
import time
from glob import glob

# from app.core.config import settings
from fastapi import HTTPException
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from pinecone import Pinecone, ServerlessSpec
from utils import logging




embeddings = UpstageEmbeddings(model="embedding-passage")

text_splitter = SemanticChunker(
    embeddings=embeddings,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
)

chunks = text_splitter.split_text(ocr_txt)
print(type(chunks))
print(len(chunks))



# LangSmith 시작
logging.langsmith(settings.LANGSMITH_PROJECT_NAME)
db_index_name = settings.PINECONE_INDEX_NAME

pc = Pinecone()
index = pc.Index(db_index_name)
embeddings = UpstageEmbeddings(model="embedding-passage")
vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

# 검색기(Retriever) 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# 청크별 검색
retrieved_docs = []    # Document list
retrieved_ids = []    # RAG id list
print("chunks len: ", len(chunks))
for chunk in chunks:
    docs = retriever.invoke(chunk)

    for doc in docs:
        doc_id = doc.id
        if not doc_id in retrieved_ids:
            retrieved_ids.append(doc_id)
            retrieved_docs.append(doc)

print("Retrieved docs len: ", len(retrieved_docs))
print("Retrieved ids len: ", len(retrieved_ids))


# 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """너는 입력을 보고 틀린 부분에 대해서 피드백을 주는 선생님이야
아래 추가 조건과 정보를 참고해서 피드백 해줘
추가 조건
1. 참고한 정보의 이름과 페이지 알려줘
2. 입력에 틀린 부분이 하나도 없을 경우에만 칭찬 한 문장 해줘
3. 다른 미사여구 없이 본론부터 말해줘

#정보:
{context}

#입력:
{question}

#답:"""
)

# 언어모델(LLM) 생성
llm = ChatUpstage(model="solar-pro", temperature=0.2)

chain = prompt | llm | StrOutputParser()

response = chain.invoke({"context": retrieved_docs, "question": ocr_txt})

print(response)




In [ ]:
def analysis_chunk(input_data):
    # LangSmith 시작
    logging.langsmith(settings.LANGSMITH_PROJECT_NAME)
    db_index_name = settings.PINECONE_INDEX_NAME

    pc = Pinecone()
    index = pc.Index(db_index_name)
    embeddings = UpstageEmbeddings(model="embedding-query")
    vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

    # 검색기(Retriever) 생성
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

    # input data 청크 나누기
    embeddings = UpstageEmbeddings(model="embedding-passage")
    text_splitter = SemanticChunker(
        embeddings=embeddings,
        breakpoint_threshold_type="percentile",
        breakpoint_threshold_amount=95,
    )
    chunks = text_splitter.split_text(input_data)
    print("input data chunk len: ", len(chunks))

    # 청크별 검색
    retrieved_docs = []    # Document list
    retrieved_ids = []    # RAG id list
    for chunk in chunks:
        docs = retriever.invoke(chunk)
        for doc in docs:
            doc_id = doc.id
            if not doc_id in retrieved_ids:
                retrieved_ids.append(doc_id)
                retrieved_docs.append(doc)
    print("Retrieved docs len: ", len(retrieved_docs))
    print("Retrieved ids len: ", len(retrieved_ids))

    # 프롬프트 생성(Create Prompt)
    prompt = PromptTemplate.from_template(
        """너는 입력을 보고 틀린 부분에 대해서 피드백을 주는 선생님이야
    아래 추가 조건과 정보를 참고해서 피드백 해줘
    추가 조건
    1. 참고한 정보의 이름과 페이지 알려줘
    2. 입력에 틀린 부분이 하나도 없을 경우에만 칭찬 한 문장 해줘
    3. 다른 미사여구 없이 본론부터 말해줘

    #정보:
    {context}

    #입력:
    {question}

    #답:"""
    )

    # 언어모델(LLM) 생성
    llm = ChatUpstage(model="solar-pro", temperature=0.2)

    # Chain 생성
    chain = prompt | llm | StrOutputParser()

    response = chain.invoke({"context": retrieved_docs, "question": input_data})

    results = {
        "rag_id": retrieved_ids,
        "response": response
    }

    return results
